In [1]:
from torch import fx
from torch.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfig, QConfigMapping, default_qconfig
from occupancy.pipelines.autoencoderkl_3d_qint8 import AutoEncoderKL3d
import torch
model = AutoEncoderKL3d(1, 1, 16)
example_inputs = torch.randn(1, 1, 256, 256, 32)
qconfig_mapping = QConfigMapping().set_global(default_qconfig)


model.encoder = prepare_fx(model.encoder, qconfig_mapping, example_inputs)
  
#model_prepared = prepare_fx(
#    model_fp32,
#    qconfig_mapping,
#    example_inputs,
#    prepare_custom_config_dict=prepare_custom_config_dict,
#)